In [47]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

import datetime


from pytz import timezone
from pycaret.classification import *

warnings.filterwarnings('ignore')

In [35]:
def get_db_connection(dbEnvironment):
    if dbEnvironment =='QA':
        try:
            # connect to SQL Server.
            SERVER = '52.44.171.130' 
            DATABASE = 'datascience' 
            USERNAME = 'nrad' 
            PASSWORD = 'ThisIsQA123' #getpass.getpass(prompt='Enter your password')
            CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
            print("The Connection to "+dbEnvironment +" was succesfull")
        except Exception as err:
            print("The Connection to "+dbEnvironment +" wasn't succesfull", err)
    elif dbEnvironment =='PRD':
        try:
            # connect to SQL Server.
            SERVER = '34.206.73.189' 
            DATABASE = 'datascience' 
            USERNAME = 'nrad' 
            PASSWORD = '83F25619-D272-4660-98A2-93AF5CC18D59' #getpass.getpass(prompt='Enter your password')
            CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
            print("The Connection to "+dbEnvironment +" was succesfull")
        except Exception as err:
            print("The Connection to "+dbEnvironment +" wasn't succesfull", err)
    return CNXN
            

In [75]:
def get_ds_scores(connection):
    try:
        
        lkupclientid = 36 # flames
        today = datetime.datetime.now()
        today_date = today.strftime("%Y-%m-%d")
        cursor = connection.cursor()
        numRecords =  f'''select count(*) as numRecords from ds.customerScores where lkupclientid ={lkupclientid} and cast(insertDate as date) = {"'"+ str(today_date) + "'"};'''
        cntTodayScores = pd.read_sql(numRecords, connection)
        cntrecords= cntTodayScores['numRecords']
        cnt_records=cntrecords[0]
        if cnt_records == 0:
            print("ds.customerScores is not updated")
        elif cnt_records != 0:
            print(cnt_records + " records been inserted in ds.customerScores table on " + today_date)
        connection.commit()
        cursor.close()
    except Exception as err:
            print("The Connection wasn't succesfull", err)


In [95]:
def get_stlr_scores(factconnection):
    try:
        
        lkupclientid = 36 # flames
        today = datetime.datetime.now()
        today_date_id = today.strftime("%Y%m%d")
        cursor = factconnection.cursor()
        fact_numRecords =  f'''select count(*) as factnumRecords from stlrflames.dw.factCustomerretentionScore where lkupclientid ={lkupclientid} and cast(dimDateId as int) = {today_date_id};'''
        fact_cntTodayScores = pd.read_sql(fact_numRecords, factconnection)
        fact_cntrecords= fact_cntTodayScores['factnumRecords']
        fact_cnt_records=fact_cntrecords[0]
        #print(fact_cnt_records)
        if fact_cnt_records == 0:
            print("dw.factCustomerretentionScore is not updated")
        elif fact_cnt_records != 0:
            print(fact_cnt_records + " records been inserted in dw.factCustomerretentionScore table on " + today_date_id)
        factconnection.commit()
        cursor.close()
    except Exception as err:
            print("The Connection wasn't succesfull", err)

In [97]:
def main():
    conn= get_db_connection("QA")
    retention_scores = get_ds_scores(conn)
    fact_retention_scores = get_stlr_scores(conn)

if __name__ == "__main__":
    main()


The Connection to QA was succesfull
ds.customerScores is not updated
dw.factCustomerretentionScore is not updated
